In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np 
import pandas as pd 
import albumentations as A  
import os 
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.signal import resample
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 让结果全部显示

import seaborn as sns
import matplotlib.pyplot as plt
import gc
import albumentations as A
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from tensorflow import random
import os 
from tensorflow.keras import initializers

# 统计代码运行的时间
from time import *
begin_time = time()

# 设置随机数
SEED = 2031

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
random.set_seed(SEED)

data_path = '/content/drive/My Drive/kesci_JZB/交子杯算法赛道—渣渣炼丹师队/code/data/'
model_path = '/content/drive/My Drive/kesci_JZB/交子杯算法赛道—渣渣炼丹师队/code/model/'
npy_path = '/content/drive/My Drive/kesci_JZB/交子杯算法赛道—渣渣炼丹师队/code/npy_file/'
pseudo_path = '/content/drive/My Drive/kesci_JZB/交子杯算法赛道—渣渣炼丹师队/code/pseudo_labels/'

In [ ]:
train = pd.read_csv(data_path + 'sensor_train.csv')
test = pd.read_csv(data_path + 'sensor_test.csv')
sub = pd.read_csv(data_path + 'result.csv')
y = train.groupby('fragment_id')['behavior_id'].min()

data = pd.concat([train, test], sort = False) # 合并数据集，默认上下拼接，不对列名进行排序。ignore_index = True重新生成索引

data1 = data[['fragment_id','behavior_id','time_point']] 
data['acc_all'] = (data.acc_x ** 2 + data.acc_y ** 2 + data.acc_z ** 2) ** .5
data['acc_allg'] = (data.acc_xg ** 2 + data.acc_yg ** 2 + data.acc_zg ** 2) ** .5
numercial_feature = [i for i in data.columns if i not in ['fragment_id','behavior_id','time_point']] # 数值型特征
# 划分数据 
train = data[data.behavior_id.isna() == False].reset_index(drop = True) # 得到训练集
test = data[data.behavior_id.isna() == True].reset_index(drop = True) # 得到测试集

# 数据标准化
sta_data1 = StandardScaler().fit_transform(data[numercial_feature])
# 数据转化为pandas 
sta_data1 = pd.DataFrame(sta_data1, columns = numercial_feature, index = data.index)
sta_data1.head() 
# 合并其他变量 
sta_data = pd.concat([data1, sta_data1], axis = 1) 
sta_data.head()

# 划分数据
sta_train = sta_data[sta_data.behavior_id.isna() == False].reset_index(drop = True) 
sta_test = sta_data[sta_data.behavior_id.isna() == True].reset_index(drop = True).drop('behavior_id', axis=1) # 由于测试集behavior_id列均为NAN，剔除

,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,acc_all,acc_allg
0,0.47866,-0.441765,0.107778,-0.082926,0.561910,0.369995,-0.237652,0.169496
1,0.15091,0.119064,-0.596496,-0.145078,0.628382,0.216928,-0.264796,-0.223509
2,0.15091,0.119064,0.389488,0.010302,0.595146,0.446528,-0.375403,0.529048
3,0.15091,-0.067879,-0.737351,-0.020774,0.628382,-0.242270,-0.141271,-1.686373
4,0.15091,0.492950,0.812052,0.010302,0.628382,0.408261,-0.002329,0.466829


,fragment_id,behavior_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,acc_all,acc_allg
0,0,0.0,27,0.47866,-0.441765,0.107778,-0.082926,0.561910,0.369995,-0.237652,0.169496
1,0,0.0,108,0.15091,0.119064,-0.596496,-0.145078,0.628382,0.216928,-0.264796,-0.223509
2,0,0.0,198,0.15091,0.119064,0.389488,0.010302,0.595146,0.446528,-0.375403,0.529048
3,0,0.0,297,0.15091,-0.067879,-0.737351,-0.020774,0.628382,-0.242270,-0.141271,-1.686373
4,0,0.0,388,0.15091,0.492950,0.812052,0.010302,0.628382,0.408261,-0.002329,0.466829


In [ ]:
## 通过采样获得相同长度的片段数据
x = np.zeros((7292, 60, 8, 1)) 
t = np.zeros((7500, 60, 8, 1)) 
for i in tqdm(range(7292)): 
  tmp = sta_train[sta_train.fragment_id == i][:]
  x[i, :, :, 0] = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id'], axis=1), 60, np.array(tmp.time_point))[0]
for i in tqdm(range(7500)): 
  tmp = sta_test[sta_test.fragment_id == i][:]
  t[i, :, :, 0] = resample(tmp.drop(['fragment_id', 'time_point'], axis=1), 60, np.array(tmp.time_point))[0]

# 图像中生成正方形黑块，参考：https://albumentations.readthedocs.io/en/latest/api/augmentations.html?highlight=cutout#albumentations.augmentations.transforms.Cutout
def data_Cutout(x, y): 
    x1 = np.zeros((x.shape[0], x.shape[1], x.shape[2], 1))
    for i in range(x.shape[0]):
      transform = A.Cutout(num_holes=8, max_h_size=1, max_w_size=1, always_apply=False, p=0.5) # Cutout 剪下8个正方形的小黑块
      x1[i, :, :, 0] = transform(image = x[i, :, :, 0])['image'] # 生成一个裁剪之后的图片
    x = np.vstack((x, x1)) # 多维数组垂直堆叠
    y = np.hstack((y, y)) # y为标签，一维数组，可以水平堆叠
    return x,y 

100%|██████████| 7500/7500 [00:14<00:00, 510.16it/s]


In [ ]:
# 线上78.8分模型 
def Net():
    input = Input(shape=(60, 8, 1)) # 单通道
    X = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(input)
    X = Activation('relu')(X)

    X = Conv2D(filters=128, kernel_size=(3, 3), padding='same')(X)
    X = Activation('relu')(X)

    X = AveragePooling2D()(X)

    X = Conv2D(filters=256, kernel_size=(3, 3), padding='same')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=512, kernel_size=(3, 3), padding='same')(X)
    X = BatchNormalization()(X) # batch正则化层最好放在激活函数之前，为了activation能更有效地使用输入信息
    X = Activation('relu')(X)

    X = GlobalAveragePooling2D()(X)
    
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    # X = GaussianNoise(0.01)(X)
    # X = Dropout(0.3)(X)
    # kernel_regularizer=regularizers.l2(0.002)
    X = Dense(19, activation='softmax')(X) 
    return Model([input], X) 

In [ ]:
pre = {} 
scores_all = [] # 存储每轮acc均值
val_train = np.zeros((7500, 19)) 
nums = 1 # nums * 25 折
cv = 25 # 一轮训练25次 
for num in range(nums): ## bagging 
    seed = np.random.randint(0, 10000) ## 每一轮的随机因子
    print('='*15, '第{}次'.format(num), '随机因子', seed, '='*15) 
    kfold = StratifiedKFold(n_splits = cv, shuffle = True) # 注：设置shuffle=True，可以不用再设置random_state了
    proba_t = np.zeros((7500, 19))
    train_pred = np.zeros((7292, 19))
    cvscores = [] # 存储每折模型对验证集评估的分数
    valid_fold = [] 
    ## 25折 
    for fold, (xx, yy) in enumerate(kfold.split(x, y)): 
        print('='*15, '第{}次'.format(num), 'fold=', fold, '='*15)
        
        y_ = to_categorical(y, num_classes=19) # 类别标签转换成二进制矩阵 
        x1, y1 = data_Cutout(x[xx], y[xx]) # 原数据(24/25 --> 7000) + 裁剪数据(7000)
        y1 = to_categorical(y1, num_classes=19)
        
        model = Net()
        model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc']) # 'Nadam'
        plateau = ReduceLROnPlateau(monitor="val_acc", verbose=0, mode='max', factor=0.10, patience=6)
        early_stopping = EarlyStopping(monitor='val_acc', verbose=0, mode='max', patience=25) # 早停可以等待周期长一些
        checkpoint = ModelCheckpoint(model_path + f'fold{fold}.h5', monitor='val_acc', verbose=0, mode='max', save_best_only=True) 
        # ModelCheckpoint的作用在每个epoch后保存模型到filepath，save_best_only=True 只保存在验证集上性能最好的模型
        model.fit( x1,y1, 
              epochs=500, 
              batch_size=32,
              verbose=1,
              shuffle=True,
              validation_data=(x[yy], y_[yy]), # 验证集取原数据集的1/25
              callbacks=[plateau, early_stopping, checkpoint] )
        
        scores = model.evaluate(x[yy], y_[yy], verbose=0) # 用训练好的模型对验证集进行评估
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100)) # model.metrics_names[1]为acc scores[0]为损失值、scores[1]为准确率
        cvscores.append(scores[1] * 100) 
        model.load_weights(model_path + f'fold{fold}.h5') # 用ModelCheckpoint保存了训练过程中的结果最好epoch的参数来预测新样本，load_weights()仅读取权重，但是需要在已有模型基础之上才可以调用
        
        ## 单模stacking
        proba_t += model.predict(t, verbose=0, batch_size=1024) / cv
        train_pred[yy] += model.predict(x[yy], verbose=0, batch_size=1024) 
        
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores))) # 对于%%，第一个%起到转义的作用，使结果输出百分号%
    scores_all.append(np.mean(cvscores))


print("综合分数：%.2f%% (+/- %.2f%%)" % (np.mean(scores_all), np.std(scores_all))) 
# 计算执行时间 
end_time = time()
run_time = (end_time - begin_time) / 60
print('该循环程序运行时间：', run_time, '分钟') 

np.save(npy_path + 'CNN_valid', train_pred)
np.save(npy_path + 'CNN_test', proba_t)

=============== 第0次 随机因子 6075 ===============
=============== 第0次 fold= 0 ===============
Epoch 1/500
438/438 [==============================] - 7s 15ms/step - loss: 1.6887 - acc: 0.4267 - val_loss: 1.8749 - val_acc: 0.3562
Epoch 2/500
438/438 [==============================] - 6s 14ms/step - loss: 1.2657 - acc: 0.5570 - val_loss: 1.5908 - val_acc: 0.4829
Epoch 3/500
438/438 [==============================] - 6s 14ms/step - loss: 1.0494 - acc: 0.6214 - val_loss: 1.1247 - val_acc: 0.5822
Epoch 4/500
438/438 [==============================] - 6s 14ms/step - loss: 0.8705 - acc: 0.6891 - val_loss: 1.1339 - val_acc: 0.5959
Epoch 5/500
438/438 [==============================] - 6s 14ms/step - loss: 0.7348 - acc: 0.7363 - val_loss: 0.9475 - val_acc: 0.7021
Epoch 6/500
438/438 [==============================] - 6s 14ms/step - loss: 0.6371 - acc: 0.7720 - val_loss: 0.8591 - val_acc: 0.7089
Epoch 7/500
438/438 [==============================] - 6s 14ms/step - loss: 0.5458 - acc: 0.8026 - val_los

acc: 79.79%
=============== 第0次 fold= 1 ===============
Epoch 1/500
438/438 [==============================] - 7s 16ms/step - loss: 1.6614 - acc: 0.4324 - val_loss: 1.7955 - val_acc: 0.4281
Epoch 2/500
438/438 [==============================] - 7s 15ms/step - loss: 1.2329 - acc: 0.5601 - val_loss: 1.3065 - val_acc: 0.5240
Epoch 3/500
438/438 [==============================] - 7s 15ms/step - loss: 1.0277 - acc: 0.6331 - val_loss: 1.4410 - val_acc: 0.5445
Epoch 4/500
438/438 [==============================] - 7s 15ms/step - loss: 0.8601 - acc: 0.6944 - val_loss: 1.1236 - val_acc: 0.6541
Epoch 5/500
438/438 [==============================] - 7s 15ms/step - loss: 0.7444 - acc: 0.7351 - val_loss: 0.8788 - val_acc: 0.7123
Epoch 6/500
438/438 [==============================] - 7s 15ms/step - loss: 0.6372 - acc: 0.7741 - val_loss: 0.9266 - val_acc: 0.7021
Epoch 7/500
438/438 [==============================] - 7s 15ms/step - loss: 0.5509 - acc: 0.8006 - val_loss: 0.9074 - val_acc: 0.7089
Epoch 

acc: 83.56%
=============== 第0次 fold= 2 ===============
Epoch 1/500
438/438 [==============================] - 7s 16ms/step - loss: 1.7014 - acc: 0.4209 - val_loss: 1.6768 - val_acc: 0.4247
Epoch 2/500
438/438 [==============================] - 7s 15ms/step - loss: 1.3037 - acc: 0.5423 - val_loss: 1.4576 - val_acc: 0.4966
Epoch 3/500
438/438 [==============================] - 7s 15ms/step - loss: 1.0766 - acc: 0.6189 - val_loss: 1.1238 - val_acc: 0.5753
Epoch 4/500
438/438 [==============================] - 7s 15ms/step - loss: 0.9180 - acc: 0.6732 - val_loss: 0.9923 - val_acc: 0.6541
Epoch 5/500
438/438 [==============================] - 7s 16ms/step - loss: 0.7989 - acc: 0.7114 - val_loss: 0.9290 - val_acc: 0.6918
Epoch 6/500
438/438 [==============================] - 7s 15ms/step - loss: 0.6983 - acc: 0.7491 - val_loss: 0.7292 - val_acc: 0.7397
Epoch 7/500
438/438 [==============================] - 6s 15ms/step - loss: 0.5963 - acc: 0.7829 - val_loss: 0.7205 - val_acc: 0.7363
Epoch 

acc: 87.33%
=============== 第0次 fold= 3 ===============
Epoch 1/500
438/438 [==============================] - 7s 16ms/step - loss: 1.6767 - acc: 0.4306 - val_loss: 2.0291 - val_acc: 0.3527
Epoch 2/500
438/438 [==============================] - 7s 15ms/step - loss: 1.2473 - acc: 0.5620 - val_loss: 1.2740 - val_acc: 0.5171
Epoch 3/500
438/438 [==============================] - 7s 15ms/step - loss: 1.0329 - acc: 0.6289 - val_loss: 1.3496 - val_acc: 0.5651
Epoch 4/500
438/438 [==============================] - 7s 15ms/step - loss: 0.8780 - acc: 0.6879 - val_loss: 1.0867 - val_acc: 0.6233
Epoch 5/500
438/438 [==============================] - 7s 15ms/step - loss: 0.7519 - acc: 0.7259 - val_loss: 0.9222 - val_acc: 0.6610
Epoch 6/500
438/438 [==============================] - 7s 15ms/step - loss: 0.6634 - acc: 0.7613 - val_loss: 0.8851 - val_acc: 0.6918
Epoch 7/500
438/438 [==============================] - 7s 15ms/step - loss: 0.5705 - acc: 0.7973 - val_loss: 0.8875 - val_acc: 0.6747
Epoch 

acc: 83.22%
=============== 第0次 fold= 4 ===============
Epoch 1/500
438/438 [==============================] - 7s 16ms/step - loss: 1.6710 - acc: 0.4292 - val_loss: 1.4668 - val_acc: 0.4863
Epoch 2/500
438/438 [==============================] - 7s 15ms/step - loss: 1.2568 - acc: 0.5553 - val_loss: 1.4194 - val_acc: 0.4795
Epoch 3/500
438/438 [==============================] - 7s 16ms/step - loss: 1.0502 - acc: 0.6264 - val_loss: 1.1457 - val_acc: 0.5788
Epoch 4/500
438/438 [==============================] - 7s 15ms/step - loss: 0.8855 - acc: 0.6851 - val_loss: 1.0019 - val_acc: 0.6336
Epoch 5/500
438/438 [==============================] - 7s 15ms/step - loss: 0.7666 - acc: 0.7234 - val_loss: 1.1643 - val_acc: 0.6062
Epoch 6/500
438/438 [==============================] - 7s 16ms/step - loss: 0.6714 - acc: 0.7616 - val_loss: 0.9400 - val_acc: 0.6644
Epoch 7/500
438/438 [==============================] - 7s 15ms/step - loss: 0.5780 - acc: 0.7904 - val_loss: 0.7938 - val_acc: 0.7500
Epoch 

...

acc: 84.19%
=============== 第0次 fold= 21 ===============
Epoch 1/500
438/438 [==============================] - 7s 16ms/step - loss: 1.6882 - acc: 0.4274 - val_loss: 1.7614 - val_acc: 0.4605
Epoch 2/500
438/438 [==============================] - 7s 15ms/step - loss: 1.2691 - acc: 0.5527 - val_loss: 1.4024 - val_acc: 0.5704
Epoch 3/500
438/438 [==============================] - 7s 15ms/step - loss: 1.0408 - acc: 0.6291 - val_loss: 1.1350 - val_acc: 0.6186
Epoch 4/500
438/438 [==============================] - 7s 15ms/step - loss: 0.8762 - acc: 0.6848 - val_loss: 1.1384 - val_acc: 0.6082
Epoch 5/500
438/438 [==============================] - 7s 16ms/step - loss: 0.7513 - acc: 0.7311 - val_loss: 0.9407 - val_acc: 0.6907
Epoch 6/500
438/438 [==============================] - 7s 15ms/step - loss: 0.6347 - acc: 0.7727 - val_loss: 0.9538 - val_acc: 0.6838
Epoch 7/500
438/438 [==============================] - 7s 16ms/step - loss: 0.5433 - acc: 0.8038 - val_loss: 0.8077 - val_acc: 0.7766
Epoch

acc: 84.54%
=============== 第0次 fold= 22 ===============
Epoch 1/500
438/438 [==============================] - 7s 16ms/step - loss: 1.6996 - acc: 0.4234 - val_loss: 2.8709 - val_acc: 0.2612
Epoch 2/500
438/438 [==============================] - 7s 15ms/step - loss: 1.2567 - acc: 0.5576 - val_loss: 1.4637 - val_acc: 0.5361
Epoch 3/500
438/438 [==============================] - 7s 15ms/step - loss: 1.0443 - acc: 0.6281 - val_loss: 1.2331 - val_acc: 0.5739
Epoch 4/500
438/438 [==============================] - 7s 15ms/step - loss: 0.8769 - acc: 0.6860 - val_loss: 0.9521 - val_acc: 0.6838
Epoch 5/500
438/438 [==============================] - 7s 15ms/step - loss: 0.7483 - acc: 0.7285 - val_loss: 0.9843 - val_acc: 0.7010
Epoch 6/500
438/438 [==============================] - 7s 15ms/step - loss: 0.6456 - acc: 0.7652 - val_loss: 0.9216 - val_acc: 0.7045
Epoch 7/500
438/438 [==============================] - 7s 15ms/step - loss: 0.5615 - acc: 0.7959 - val_loss: 0.8330 - val_acc: 0.7388
Epoch

acc: 82.47%
=============== 第0次 fold= 23 ===============
Epoch 1/500
438/438 [==============================] - 7s 16ms/step - loss: 1.6886 - acc: 0.4235 - val_loss: 1.7331 - val_acc: 0.3883
Epoch 2/500
438/438 [==============================] - 7s 15ms/step - loss: 1.2888 - acc: 0.5468 - val_loss: 1.7802 - val_acc: 0.4364
Epoch 3/500
438/438 [==============================] - 7s 15ms/step - loss: 1.0634 - acc: 0.6228 - val_loss: 1.6438 - val_acc: 0.5326
Epoch 4/500
438/438 [==============================] - 7s 15ms/step - loss: 0.8981 - acc: 0.6826 - val_loss: 1.1871 - val_acc: 0.5979
Epoch 5/500
438/438 [==============================] - 7s 15ms/step - loss: 0.7697 - acc: 0.7242 - val_loss: 1.2476 - val_acc: 0.5636
Epoch 6/500
438/438 [==============================] - 7s 16ms/step - loss: 0.6747 - acc: 0.7572 - val_loss: 0.9942 - val_acc: 0.6804
Epoch 7/500
438/438 [==============================] - 7s 15ms/step - loss: 0.5811 - acc: 0.7894 - val_loss: 0.9721 - val_acc: 0.7285
Epoch

acc: 81.44%
=============== 第0次 fold= 24 ===============
Epoch 1/500
438/438 [==============================] - 7s 16ms/step - loss: 1.6774 - acc: 0.4253 - val_loss: 2.0686 - val_acc: 0.3265
Epoch 2/500
438/438 [==============================] - 7s 15ms/step - loss: 1.2652 - acc: 0.5550 - val_loss: 1.3681 - val_acc: 0.5533
Epoch 3/500
438/438 [==============================] - 7s 15ms/step - loss: 1.0472 - acc: 0.6231 - val_loss: 1.1698 - val_acc: 0.5979
Epoch 4/500
438/438 [==============================] - 7s 15ms/step - loss: 0.8844 - acc: 0.6783 - val_loss: 0.9594 - val_acc: 0.6632
Epoch 5/500
438/438 [==============================] - 7s 15ms/step - loss: 0.7557 - acc: 0.7280 - val_loss: 0.9420 - val_acc: 0.6598
Epoch 6/500
438/438 [==============================] - 7s 16ms/step - loss: 0.6532 - acc: 0.7692 - val_loss: 0.9720 - val_acc: 0.6735
Epoch 7/500
438/438 [==============================] - 7s 15ms/step - loss: 0.5590 - acc: 0.7973 - val_loss: 0.9553 - val_acc: 0.7113
Epoch

acc: 82.47%
83.54% (+/- 1.79%)
综合分数：83.54% (+/- 0.00%)
该循环程序运行时间： 151.01849813461303 分钟


In [ ]:
########### 生成"伪标签"用于其他模型的训练 ############################
proba = np.load(npy_path + 'CNN_test.npy')
res_sub = pd.read_csv(data_path + 'result.csv') 
## proba_t为预测的测试集的y
jiandu = np.max(proba, axis=1)  

## 记录大于0.95的位置（共3574个）索引值正好对应fragment_id（0-7500）
list_num = []
for i in range(len(jiandu)):
  if jiandu[i]>0.95:
    list_num.append(i)

## 创建成pandas文件
Semi_supervision = pd.DataFrame(list_num, columns = ['fragment_id'])
test = pd.read_csv(data_path + 'sensor_test.csv')
res_sub.behavior_id = np.argmax(proba, axis=1) # 获取预测的行为Label

## 为了方便，读取预测文件进行merge
result = res_sub
Semi_supervision = Semi_supervision.merge(result, how='left', on='fragment_id') # 测试集预测概率大于95的对应片段的行为Label

## 将label merge到test中，作为测试集的真实标签，因而这部分数据可以作为新的训练数据
sps = test.merge(Semi_supervision, how='left', on='fragment_id')

## 将test的非空部分读取出来的就是新增的数据集
other = sps[~sps['behavior_id'].isna()]

other['behavior_id'] = other['behavior_id'].astype('int16')
other['fragment_id'] += 10000 ## 为了使新增数据更有区分度
other.to_csv(pseudo_path + 'pseudo_labels.csv',index=False) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
other

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
58,10001,151,0.1,0.0,0.0,-1.0,4.6,8.4,3
59,10001,232,0.0,0.0,0.1,-0.9,4.4,8.4,3
60,10001,318,0.0,0.0,0.0,-1.2,4.6,8.3,3
61,10001,406,0.2,-0.1,0.0,-0.8,4.4,8.2,3
62,10001,493,0.1,0.0,-0.3,-0.8,4.4,8.2,3
...,...,...,...,...,...,...,...,...,...
430121,17498,4547,-0.1,-0.1,0.3,-0.5,0.6,9.9,12
430122,17498,4635,0.3,0.3,-0.1,-0.2,1.2,9.4,12
430123,17498,4721,0.2,0.2,-0.1,-0.2,1.1,9.7,12
430124,17498,4810,0.1,0.0,0.3,-0.3,0.8,10.0,12
